# Cálculo do TF-IDF para os termos dos textos descritivos

## Tokenização dos textos

In [1]:
# Imports
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 

import pandas as pd

#import spacy
import string

In [2]:
## Extraindo frequencias de termos dos textos descritivos do dataset.

In [3]:
df = pd.read_json('data_files/merged_datasets.json', orient='split')

In [4]:
corpus = df['Text']

In [5]:
# Apagando a variável df porque não será mais utilizada.
del df

In [6]:
# Quebrando cada texto em uma lista de sentenças.
corpus_sent_tokenized = [sent_tokenize(texto) for texto in corpus]

In [7]:
# Apagando a variável corpus porque não será mais utilizada.
del corpus

In [8]:
# Quebrando as sentenças em palavras, de modo que cada sentença seja transformada em lista de palavras.
corpus_word_tokenized = [[word_tokenize(sent) for sent in text] for text in corpus_sent_tokenized]

In [9]:
# Apagando a variável corpus_sent_tokenized porque não será mais utilizada
del corpus_sent_tokenized

## Remoção das stopwords e pontuações

In [10]:
english_stops = set(stopwords.words('english'))

In [11]:
def remove_stopwords_e_pontuacao(lista_words):
    tokens_limpos = [word.lower() for word in lista_words if (word.lower() not in english_stops and word not in string.punctuation)]
    return tokens_limpos

In [12]:
corpus_limpo = [[remove_stopwords_e_pontuacao(sent) for sent in text] for text in corpus_word_tokenized]

In [13]:
# Apagando a variável corpus_word_tokenized porque não será mais utilizada.
del corpus_word_tokenized

In [14]:
# Faz-se necessário juntar todas as palavras de cada texto em uma lista única, para que seja feita a contagem
def abre_listas(lista_de_listas):
    lista_dummy = []
    for lista in lista_de_listas:
        for elemento in lista:
            lista_dummy.append(elemento)
    return lista_dummy

In [15]:
doc = abre_listas(corpus_limpo[0])

In [16]:
documentos = [abre_listas(doc) for doc in corpus_limpo]

In [17]:
# Apagando a variável corpus_word_tokenized porque não será mais utilizada.
del corpus_limpo

## Stemming e Lemmatization

### Stemming

In [18]:
from nltk.stem import PorterStemmer
from nltk.stem import RegexpStemmer

In [19]:
porter_stemmer = PorterStemmer()

In [20]:
porter_stemmer.stem('cooking')

'cook'

In [21]:
porter_stemmer.stem('cookery')

'cookeri'

In [22]:
regexp_stemmer = RegexpStemmer('ing')

In [23]:
porter_stemmer.stem(regexp_stemmer.stem('cooking'))

'cook'

In [24]:
porter_stemmer.stem(regexp_stemmer.stem('activation'))

'activ'

In [25]:
porter_stemmer.stem(regexp_stemmer.stem('activating'))

'activat'

In [26]:
# Aplicando stemming, isto é, retirada de sufixos, para cada palavra na lista de documentos
docs_stemmed = [[porter_stemmer.stem(regexp_stemmer.stem(word)) for word in doc] for doc in documentos]

In [27]:
len(docs_stemmed)

3321

### Lemmatization

In [28]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /home/rafael/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
wordnet_lemmatizer = WordNetLemmatizer()

In [30]:
print(wordnet_lemmatizer.lemmatize('mice'))
print(wordnet_lemmatizer.lemmatize('cacti'))  # plural da palavra cactus - cactuses (inglês) ou cacti (latin)
print(wordnet_lemmatizer.lemmatize('horses'))
print(wordnet_lemmatizer.lemmatize('wolves'))

mouse
cactus
horse
wolf


In [31]:
# Aplicando lemmatization, isto é, a aglutinação das palavras com o mesmo radical comum.
docs_stemmed_lemmatized = [[wordnet_lemmatizer.lemmatize(word) for word in doc] for doc in documentos]

## Cálculo do TF-IDF

### Cálculo do TF

In [32]:
# Função para calcular Term-frequency (frequencia do termo)
def TermFreq(documento, palavra):
    doc_length = len(documento)
    ocorrencias = len([w for w in documento if w == palavra])
    return ocorrencias / doc_length

In [33]:
TermFreq(docs_stemmed_lemmatized[0], 'cyclin')

0.02197530864197531

In [34]:
TermFreq(docs_stemmed_lemmatized[0], 'cancer')

0.002962962962962963

In [35]:
palavras_corpus = abre_listas(docs_stemmed_lemmatized)

In [36]:
# Criação de uma lista única contendo todas as palavras dos documentos para contagem dos termos no corpus
set_palavras_corpus = set(palavras_corpus)

In [37]:
len(set_palavras_corpus)

276083

In [38]:
len(palavras_corpus)

21194984

### Estabelecendo um contador para apurar a frequência das palavras

In [39]:
from collections import Counter

In [40]:
%%time
contador = Counter(palavras_corpus)

CPU times: user 2.63 s, sys: 4.05 ms, total: 2.64 s
Wall time: 2.63 s


In [41]:
len(contador)

276083

In [42]:
contador['revealed']

12185

In [43]:
chaves = contador.keys()

In [44]:
'kinase' in chaves

True

In [45]:
# Versão 1 implementada totalmente em Python

# Criamos um corpus Bag of words
def cria_dict(set_palavras, lista_docs):
    output = {}
    for word in set_palavras:
        output[word] = 0
        for doc in lista_docs:
            if word in doc:
                output[word] += 1
    return output

In [46]:
lista_teste = docs_stemmed_lemmatized[0:10]

In [47]:
set_teste = set(abre_listas(lista_teste))

In [48]:
%%time
dict_1 = cria_dict(set_teste, lista_teste)

CPU times: user 3.38 s, sys: 5 µs, total: 3.38 s
Wall time: 3.38 s


In [49]:
lista_sets = [Counter(doc).keys() for doc in docs_stemmed_lemmatized]

In [50]:
lista_teste_2 = lista_sets[0:10]

In [51]:
# Versão 2 otimizada utilizando contadores.

# Criamos um corpus Bag of words
def cria_dict_2(set_palavras, lista_sets):
    output = {}
    for word in set_palavras:
        output[word] = 0
        for set in lista_sets:
            if word in set:
                output[word] += 1
    return output

In [52]:
%%time
dict_2 = cria_dict_2(set_teste, lista_teste_2)

CPU times: user 12.4 ms, sys: 1 µs, total: 12.4 ms
Wall time: 12.1 ms


A função cria_dict_2 apresentou desempenho 340x melhor do que a função cria_dict

In [53]:
%%time
# Criação do dicionário completo com todos os documentos do dataset
dict_completo = cria_dict_2(set_palavras_corpus, lista_sets)

CPU times: user 2min 7s, sys: 11.8 ms, total: 2min 7s
Wall time: 2min 7s


In [54]:
len(dict_completo)

276083

### Cálculo do IDF

In [55]:
# Função para calcular a Frequência Inversa de Documentos
def InverseDocumentFrequency(word):
    N = len(docs_stemmed_lemmatized)
    try:
        df = dict_completo[word] + 1
    except:
        df = 1
    return np.log(N/df)

In [56]:
InverseDocumentFrequency('human')

0.09467803999007461

In [57]:
InverseDocumentFrequency('mother')

2.543500814054053

In [58]:
InverseDocumentFrequency('data')

0.031195190077938376

### Cálculo do TF-IDF

In [59]:
# Função TF-IDF
def TFIDF(doc, word):
    tf = TermFreq(doc, word)
    idf = InverseDocumentFrequency(word)
    return tf * idf

In [60]:
TFIDF(docs_stemmed_lemmatized[0], 'cellular')

0.00029136573894505155

In [61]:
TFIDF(docs_stemmed_lemmatized[0], 'activating')

0.0002700738922698808

In [62]:
for word in set(docs_stemmed_lemmatized[0]):
    print(f'{word} - TF-IDF: {TFIDF(docs_stemmed_lemmatized[0], word)}')

novagen - TF-IDF: 0.000821950056361782
blot - TF-IDF: 0.002506108389917302
pair1 - TF-IDF: 0.0018308330964979759
cdi1 - TF-IDF: 0.0018308330964979759
unrelated - TF-IDF: 0.0007269504521926421
purified - TF-IDF: 0.0008598332513155348
involving - TF-IDF: 0.00017033096262961939
determine - TF-IDF: 7.520646035174785e-05
contrary - TF-IDF: 0.0010731491485220538
positively - TF-IDF: 0.0015306897961130646
disease - TF-IDF: 6.339487018934889e-05
s4a - TF-IDF: 0.0006054393606313318
interact - TF-IDF: 0.0015041237546106485
activated - TF-IDF: 0.00013580895378912115
embryonic - TF-IDF: 0.00027690241036304193
decrease - TF-IDF: 0.0007010835836111422
compromised - TF-IDF: 0.00047049029733057385
14 - TF-IDF: 9.56883667329904e-05
β-glycerophosphate - TF-IDF: 0.0017347661092553862
syndrome-associated - TF-IDF: 0.0021700983479863898
phosphorylates - TF-IDF: 0.0010464492861663622
biological - TF-IDF: 0.0005280842035661193
existence - TF-IDF: 0.0007467855345261407
aberrant - TF-IDF: 0.0004735257396638717

figure - TF-IDF: 6.152155968898984e-05
dusp9 - TF-IDF: 0.0036616661929959517
notably - TF-IDF: 0.0002253910867155649
revealed - TF-IDF: 0.0002860157524010767
amplicons - TF-IDF: 0.0005418563495650559
explanation - TF-IDF: 0.0002728285833556258
characterization - TF-IDF: 0.0003712302937412336
s10a - TF-IDF: 0.002414392788399282
confers - TF-IDF: 0.00035394069865736315
work - TF-IDF: 0.0003356484084868842
staining - TF-IDF: 0.00018435698577762914
associated - TF-IDF: 0.0001274319511334636
b - TF-IDF: 0.00025890326009408483
supernatant - TF-IDF: 0.0006805911856233439
-producing - TF-IDF: 0.0016596856445078658
film - TF-IDF: 0.0005961207611293943
cyclin-box-fold - TF-IDF: 0.0018308330964979759
deorphanize - TF-IDF: 0.0018308330964979759
alone - TF-IDF: 0.0008707950591142504
homogeneous - TF-IDF: 0.001252280093431596
coktail - TF-IDF: 0.0018308330964979759
gst-cdk10kd - TF-IDF: 0.0018308330964979759
defect - TF-IDF: 0.0002228113691275019
proline - TF-IDF: 0.00043602464263101115
sectionnext 

## Verificando quantas palavras aparecem menos vezes no corpus todo
Intenção é eliminar estes valores espúrios, uma vez que não servirão para nenhuma generalização do modelo

In [63]:
len(contador)

276083

In [64]:
%%time
eliminar = [word for word in contador.keys() if contador[word] == 1]

CPU times: user 79.6 ms, sys: 0 ns, total: 79.6 ms
Wall time: 79 ms


In [65]:
eliminar

['syndrome.cyclin-dependent',
 'deorphanize',
 'cells.previous',
 'sectionresultsa',
 'cdi1',
 'kap',
 '1.cdk10',
 'peg202',
 'pjg4-5',
 's1d.next',
 'cdk10.p2',
 'cells.to',
 'gst-cdk10/strepii-cyclin',
 '/strepii-cyclin',
 '2.cdk10',
 'str-cycm',
 'substrate.cdk10',
 'ets2.fig',
 '3.cyclin',
 '0.001.we',
 '-cdk10',
 '4.cdk10/cyclin',
 's5–s8',
 'ser220/ser225',
 'flag-ets2dbm',
 'apc-cdh1–mediated',
 'serines.finally',
 'c.555+1g',
 '5.decreased',
 's10c.previous',
 'sectiondiscussionin',
 'pregenomic',
 'functions.the',
 'stages.we',
 'dsmcpas',
 'non–proline-directed',
 'docking-mediated',
 'apc-cdh1',
 'studies.our',
 'intringuingly',
 'proliferation.previous',
 'methodscloning',
 'methods.yeast',
 '.mammalian',
 'plasmids/sirnas',
 'h.coimmunoprecipitation',
 'phenylphosphate',
 'coktail',
 'r960',
 'pab0847p',
 'anti-cycm',
 'pab0882-p',
 'anti-ets2',
 'a5060',
 '170–6516',
 '172–1019',
 'proteins.gst-cdk10',
 '/strepii-cycm',
 'bacmids',
 'gst-cdk10-',
 'gst-cdk10kd',
 'strepii

In [66]:
len(eliminar)

105919

In [67]:
from tqdm import tqdm

In [68]:
# Verificando a qtd de palavras que podem ser eliminadas que aparecem até 10x no corpus.
eliminar = [[word for word in contador.keys() if contador[word] <= n+1] for n in range(10)]

In [69]:
for n in range(len(eliminar)):
    print(f'Número de palavras que aparecem até {n+1} vez-es: {len(eliminar[n])}')

Número de palavras que aparecem até 1 vez-es: 105919
Número de palavras que aparecem até 2 vez-es: 149547
Número de palavras que aparecem até 3 vez-es: 168673
Número de palavras que aparecem até 4 vez-es: 184821
Número de palavras que aparecem até 5 vez-es: 193744
Número de palavras que aparecem até 6 vez-es: 201524
Número de palavras que aparecem até 7 vez-es: 206168
Número de palavras que aparecem até 8 vez-es: 211748
Número de palavras que aparecem até 9 vez-es: 215623
Número de palavras que aparecem até 10 vez-es: 218927


In [70]:
# Eliminando palavras que aparecem 10 vezes ou menos em todo o corpus
docs_compressed = [list(set(doc) - set(eliminar[9])) for doc in tqdm(docs_stemmed_lemmatized)]

100%|██████████| 3321/3321 [01:53<00:00, 29.23it/s]


In [71]:
len(docs_compressed)

3321

In [78]:
## Construindo um dicionário para armazenar os resultados do tf_idf por palavra, por documento

In [79]:
%%time
dict_tf_idf = {c: {word: TFIDF(doc, word) for word in doc} for (c, doc) in enumerate(docs_compressed)}

CPU times: user 5min 1s, sys: 143 ms, total: 5min 1s
Wall time: 5min 1s


In [80]:
dict_tf_idf[0]

{'200': 0.0007600033503400419,
 'assay': 7.38197822716745e-05,
 'role': 0.00013094698689192474,
 'reveal': 0.0007004142122325023,
 'novagen': 0.0027488833429109966,
 'untagged': 0.003356705163948965,
 'predicted': 0.00030084059719487447,
 'migration': 0.0012453763188801897,
 'especially': 0.0007009951224439685,
 '1i': 0.003401351919746055,
 'hek293': 0.0019450442465327113,
 'blot': 0.0004930169028593322,
 'supersignal': 0.002806350966196872,
 'activates': 0.0009027306937251871,
 'inactivation': 0.0007233781087629961,
 'confirming': 0.0010231269782915277,
 'unrelated': 0.0012155860162593726,
 'c-20': 0.003416787207121903,
 'zebrafish': 0.0028213652676693664,
 'absence': 0.0002515162381705853,
 'molecular': 0.00017033336321767333,
 'involving': 0.0005696452507431532,
 'purified': 0.0005751155520772776,
 'up-regulation': 0.0018509935296808005,
 'determine': 0.0002515162381705853,
 'ratio': 0.00048090936939323785,
 'verified': 0.0008982910925390762,
 'nitrocellulose': 0.0014409522030027096

In [81]:
import json

with open('data_files/data_tf_idf.json', 'w') as fp:
    json.dump(dict_tf_idf, fp)
    
fp.close()